In [ ]:
!pip install ortools
!pip install dtale

In [2]:
import pandas as pd
import numpy as np
import collections
#from ortools.sat.python import cp_model
#from dtale import show
#import dtale.app as dtale_app
#dtale_app.USE_COLAB = True
# Clone the entire repo.
!git clone -l -s https://github.com/mswastik/optimization.git cloned-repo
%cd cloned-repo

Cloning into 'cloned-repo'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 71 (delta 18), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (71/71), done.
/content/cloned-repo


In [3]:
df=pd.read_excel('orders.xlsx')
ma=pd.read_excel('item master.xlsx')
bu=pd.read_excel('item master.xlsx',sheet_name='bulk')
pa=pd.read_excel('item master.xlsx',sheet_name='packing')

In [4]:
df = df[df['Line']=='Tmmthpkhti Limi']
df = df[df['Plant']==1024]
del df['Line']
df['due'] = df['Dispatch Date'] - df['SO Date'].min()
df['due'] = df['due'].dt.days
df['sodate'] = df['SO Date'] - df['SO Date'].min()
df['sodate'] = df['sodate'].dt.days
df.reset_index(inplace=True,drop=True)
df['Key']=df['SO'].astype('str')+df['FG Code']
del df['Dispatch Date']
del df['SO Date']
del df['Description']
del df['Customer']

In [26]:
jobs =df['Key']
op =[1,2,3,4]
mc = [1,2,3,4,5,6]
horizon = max(df["due"])+100
due = df.set_index('Key')
dur=df[['Key','FG Code','Order Qty']]
dur=dur.merge(pa[["FG Code","Line","Output"]], on="FG Code")
dur["duration"]=dur["Order Qty"]/dur["Output"]
del dur["Order Qty"]
dur.set_index(['Key','Line'],inplace=True)
del dur['FG Code']
del dur['Output']

In [77]:
dur[dur['duration']>100]

duration
Key                   Line          
17023287gd29907800iX  4          864
17023434gd29907800iX  4          864
17023678gd29907800iX  4          864
17023507gd29907800iX  4         1152
17023287gd39000800iX  1        10000
...                              ...
17023975gd57812500iX  3         1012
                      4         1012
                      5         1012
                      6         1012
Jum19plkmgd23005001dm 4         3082

[319 rows x 1 columns]

In [79]:
800/24

33.333333333333336

In [27]:
dur['duration']=dur['duration'].fillna(0).astype('int')

In [13]:
from dtale import show
import dtale.app as dtale_app
dtale_app.USE_COLAB = True
#show(jobs)

https://n6ekujl3anh-496ff2e9c6d22116-40000-colab.googleusercontent.com/dtale/main/1

In [8]:
from ortools.sat.python import cp_model
model = cp_model.CpModel()

In [67]:
sch = dict.fromkeys([1,2,3,4,5,6],[])
delay = model.NewIntVar(0, horizon, 'delay')
for j in jobs:
    for m in mc:
        suffix = '_%s_%i' % (j,m)
        start = model.NewIntVar(0, horizon, "start"+suffix)
        end = model.NewIntVar(0, horizon, "end"+suffix)
        duration = model.NewIntervalVar(start, dur.loc[(j,m)].values[0], end, "duration"+suffix)
        #delay += due['due'].loc[j]-end
        #model.Add(due['due'].loc[j]-end)
        sch[m].append(duration)
    delay += due['due'].loc[j]-end

In [ ]:
for j in jobs:
        model.Add(sch[m])

In [68]:
for m in mc:
        model.AddNoOverlap(sch[m])
model.Minimize(delay)

In [69]:
solver = cp_model.CpSolver()
solver.Solve(model)
solver.StatusName()

'INFEASIBLE'

**Indices**  
j - job  
m - machine  

**Decision Variable**  
$s_{j,m}$ - start time of job j at machine m  
$e_{j,m}$ - end time of job j at machine m

**Parameters**  
H - Horizon  
$d_j$ - delivery date of job j  
$q_j$ - order qty of job j  

**Objective**  
$ \sum_{j} d_j - max(e_{j,m})  \qquad \forall m \in mc$  

**Constraints**  
